In [13]:
import h5py

import numpy as np

from shutil import copyfile #safer than using sys

In [14]:
def cart2pol(x, y):
    '''
    cart2pol(x, y)
    
    x: x values
    y: y values
    
    Will convert cartesian coordinates into polar
    
    returns:
    r, theta
    '''
    r = np.sqrt(x**2 + y**2)
    theta = np.arctan2(y, x)
    return r, theta


def pol2cart(r, theta):
    '''
    pol2cart(r, theta)
    
    r: r values
    theta: theta values
    
    Will convert polar coordimates into cartesian
    
    returns:
    x,y
    '''
    x = np.multiply(r,np.cos(theta))
    y = np.multiply(r,np.sin(theta))
    return x, y 

def convert_coordinates(x, y, center=None):
    '''
    x is detector coordinate x
    y is detector coordinate y
    
    center is where the bem goes 
    through the detector in format (x,y)
    
    returns:
    
    r:      distance from center in pixels
    theta:  angle in the detector plane of x,y, measured counter-
            clockwise. 0 is at x larger than center[0], 
            and y == center[1]
    '''
    if center==None:
        error=True 
        #ADD SOME ERROR OR WARNING HERE LATER
    else:
        x_offset=np.subtract(x,center[0])
        y_offset=np.subtract(y,center[1])
        r,theta=cart2pol(x_offset,y_offset)
    return r,theta


    
def pos2q(r,wavelength_AA,distance_to_detector_m,pixel_size_m):
    '''
    r: distance in pixels from center, scalar or array
    wavelength_AA: wavelength in Angstrom
    distance_to_detector_m: distance to the center in meters
    pixel_size_m: size of a pixel in m
    '''
    
    r_array = np.atleast_1d(np.asarray(r))
    q = np.zeros(np.shape(r_array))   
    
    # 2d to q
    # q=2*pi/d or d= (2*pi)/q , 2 d = 4 pi / q

    # BRAGGS LAW:
    # 2*d sin(theta) = lambda
    # OR written as:
    # 4 pi sin(theta)/q = lambda
    # q = 4 pi sin(theta)/lambda

    # reflection angle noted with alpha
    # Also considering that theta = alpha /2 :
    # q = 4 pi sin(alpha/2)/lambda

    # distance from center = x
    # distance to detector = y
    # Also considering that tan(alpha) = x/y:
    # alpha= arctan(x/y)
    # q = 4 pi sin(arctan(x/y)/2)/lambda

    for r_index, r in enumerate(r_array):
        r_m = r*pixel_size_m

        # fraction of distance from center and distance to the detector
        fraction = np.divide(r_m, distance_to_detector_m)

        # the scattering angle, note that it's half the angle from the
        # reflection to the beam at the interaction point
        scattering_angle = np.arctan(fraction)/2.0

        #
        # in reverse Angstrom
        q[r_index] = 4*np.pi * \
            np.divide(np.sin(scattering_angle), wavelength_AA)

    return(q)

def add_q_to_new_hdf5(in_file_name,out_file_name,detector_x_size=1760,detector_y_size=1760):
    '''
    add_q_to_new_hdf5(in_file_name,out_file_name,detector_x_size=1760,detector_y_size=1760)
    
    in_file_name : path and filename of input, will be left in place
    out_file_name : will be created as a copy of in_file_name, with added datasets with q 
    
    both files are closed.
    '''


    f = h5py.File(in_file_name, 'r')
    copyfile(in_file_name, out_file_name)
    f_out = h5py.File(out_file_name, 'a')


    list_of_shots = [x for x in f['shots'].keys()]
    
    #iterate through all shots
    for shot_index, shot in enumerate(list_of_shots):
        x_size = detector_x_size
        y_size = detector_y_size
        center_of_detector_pixels = [np.divide(x_size, 2), np.divide(y_size, 2)]
        distance_to_detector_m=f["LCLS"]["detectorDistanceInMeterMean"]
        wavelength_AA=f["LCLS"]["wavelengthInAngstromMean"]
        pixel_size=np.multiply(f["LCLS"]["detectorPixelSizeInMicron"],1e-6)
        numberOfPeaks=f['shots'][shot]['numberOfPeaks'][()]
        
        
        if numberOfPeaks > 0:
            # If peaks are found, convert to polar
            
            polar_list = [[convert_coordinates(x,y,center_of_detector_pixels)[0],
                  convert_coordinates(x,y,center_of_detector_pixels)[1]] 
                 for x, y in f['shots'][shot]['peakCenterOfMassInPixels']]
            q=np.zeros(len(range(numberOfPeaks)))
            qx=np.zeros(len(range(numberOfPeaks)))
            qy=np.zeros(len(range(numberOfPeaks)))
            angle=np.zeros(len(range(numberOfPeaks)))
            
            #iterate over all peaks
            for peak in range(numberOfPeaks):
                polar=polar_list[peak]
                print(polar)
                #convert polar in pixels to momentum transfer
                q_value=(pos2q(polar[0],wavelength_AA,distance_to_detector_m,pixel_size))

                #angle is the same in pixel and q space
                angle_value=polar[1] 
                
                #converting back to cartesian
                qx_value,qy_value=pol2cart(q_value,angle_value)

                
                #these are for the list per shot
                angle[peak]=angle_value
                q[peak]=q_value                 
                qx[peak]=qx_value
                qy[peak]=qy_value
                
                
                #adding datasets into the peaks
                f_out.create_dataset('shots/{0}/peak{1}/peakCenterOfMassQMagnitude'.format(shot,peak),data=q_value)
                f_out.create_dataset('shots/{0}/peak{1}/peakCenterOfMassQAngle'.format(shot,peak),data=angle_value)
                f_out.create_dataset('shots/{0}/peak{1}/peakCenterOfMassQx'.format(shot,peak),data=qx_value)
                f_out.create_dataset('shots/{0}/peak{1}/peakCenterOfMassQy'.format(shot,peak),data=qy_value)
            #adding lists to the shots   
            f_out.create_dataset('shots/' + shot + '/peakCenterOfMassQMagnitude',data=q)
            f_out.create_dataset('shots/' + shot + '/peakCenterOfMassQAngle',data=angle)
            f_out.create_dataset('shots/' + shot + '/peakCenterOfMassQx',data=qx)            
            f_out.create_dataset('shots/' + shot + '/peakCenterOfMassQy',data=qy)

    f.close()
    f_out.close()


## EXAMPLE OF USAGE

In [12]:
add_q_to_new_hdf5(in_file_name='data_for_examples/r0144_type1-peaks.h5',out_file_name='data_for_examples/r0144_type1-peaks-q.h5')

[898.2420918106324, -1.0434736945320657]
[515.1146894931909, -1.6789556720622585]
[455.25990924339567, -2.795530387113197]
[511.71663153304644, -2.9924232600939273]
[720.3504760062718, 2.922366399510713]
[510.8701065458394, 0.7880403780086962]
[469.5453019990659, 2.269282482683121]
[504.0841908497114, 1.5479838334998985]
[661.5670666210686, 1.7583660691665546]
[845.6382465299259, -1.7012034742364943]
[1095.8173231557007, -2.302472729780134]
[497.8555964237214, -2.359333391781559]
[444.7269006940462, -0.75557720351535]
[796.4118764595199, -0.048432324311380184]
[501.67187158727285, 2.95042734920642]
[464.6067949629244, 1.7560129496664592]
[801.6326117396743, 0.6235293207800132]
[769.9891599303095, 0.6477857993817872]
[784.8356368326212, 0.636007393352385]
[857.8045678333515, 0.5980585485119458]
[862.7788027343047, 2.301522348682121]
[698.4528286625045, 1.2555036526297576]
[545.0028349382729, -1.3545011852807651]
[492.7121495390271, -1.6312115976755497]
[544.2537777141523, -2.46732140995